In [ ]:
!pip install -q openai-whisper
print("✓ Whisper installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✓ Whisper installed successfully!


In [ ]:
from google.colab import files
import shutil
import os

# Create directory for audio files
os.makedirs('audio_files', exist_ok=True)

# Upload MP3 file(s)
print("Please select your MP3 file(s) to upload:")
uploaded = files.upload()

# Move uploaded files to audio_files directory
for filename in uploaded.keys():
    shutil.move(filename, f'{filename}')
    print(f"✓ Uploaded: {filename}")

Please select your MP3 file(s) to upload:


Saving 15.mp3 to 15.mp3
✓ Uploaded: 15.mp3


In [ ]:
import whisper
import os
from pathlib import Path
import json
from datetime import datetime


# base / small / medium / large
ukuran_model = "large"
nama_file = filename

def transcribe_indonesian_audio(audio_path, model_size = ukuran_model, output_format="txt"):
    """
    Transcribe Indonesian audio file to text using Whisper

    Parameters:
    - audio_path: Path to the MP3 file
    - model_size: Whisper model size ('tiny', 'base', 'small', 'medium', 'large')
                  larger = more accurate but slower
    - output_format: Output format ('txt', 'json', 'both')

    Returns:
    - Dictionary containing transcription results
    """

    print(f"Loading Whisper model ({model_size})...")
    model = whisper.load_model(model_size)

    print(f"Transcribing: {audio_path}")
    print("This may take a few minutes depending on the audio length...")

    # Transcribe with Indonesian language specified
    result = model.transcribe(
        audio_path,
        language="id",  # Indonesian language code
        verbose=True
    )

    return result

def save_transcription(result, audio_path, output_format="txt"):
    """
    Save transcription results to file(s)
    """
    base_name = Path(audio_path).stem
    output_dir = Path("transcriptions")
    output_dir.mkdir(exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save as text file
    if output_format in ["txt", "both"]:
        txt_path = output_dir / f"{base_name}_{timestamp}.txt"
        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(result["text"])
        print(f"✓ Transcription saved to: {txt_path}")

    # Save as JSON with detailed information
    if output_format in ["json", "both"]:
        json_path = output_dir / f"{base_name}_{timestamp}.json"

        # Prepare detailed output
        detailed_result = {
            "audio_file": str(audio_path),
            "timestamp": timestamp,
            "language": result.get("language"),
            "full_text": result["text"],
            "segments": [
                {
                    "id": seg["id"],
                    "start": seg["start"],
                    "end": seg["end"],
                    "text": seg["text"]
                }
                for seg in result.get("segments", [])
            ]
        }

        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(detailed_result, f, ensure_ascii=False, indent=2)
        print(f"✓ Detailed transcription saved to: {json_path}")

    return result["text"]

def transcribe_directory(directory_path, model_size= ukuran_model, output_format="txt"):
    """
    Transcribe all MP3 files in a directory
    """
    directory = Path(directory_path)
    mp3_files = list(directory.glob("*.mp3"))

    if not mp3_files:
        print(f"No MP3 files found in {directory_path}")
        return

    print(f"Found {len(mp3_files)} MP3 file(s) to transcribe\n")

    results = {}

    for i, mp3_file in enumerate(mp3_files, 1):
        print(f"\n{'='*60}")
        print(f"Processing file {i}/{len(mp3_files)}: {mp3_file.name}")
        print(f"{'='*60}")

        try:
            result = transcribe_indonesian_audio(str(mp3_file), model_size)
            text = save_transcription(result, str(mp3_file), output_format)
            results[mp3_file.name] = text

            print(f"\nTranscription preview:")
            print(f"{text[:200]}..." if len(text) > 200 else text)

        except Exception as e:
            print(f"✗ Error transcribing {mp3_file.name}: {str(e)}")
            results[mp3_file.name] = f"Error: {str(e)}"

    return results

def main():
    """
    Main function - customize this for your use case
    """

    # Option 1: Transcribe a single file
    print("=== Indonesian Audio Transcription ===\n")

    # Example: Single file transcription
    audio_file = nama_file  # Change this to your file path

    if os.path.exists(audio_file):
        print("Mode: Single file transcription")
        result = transcribe_indonesian_audio(audio_file, model_size= ukuran_model)
        save_transcription(result, audio_file, output_format="both")
    else:
        # Option 2: Transcribe all MP3 files in a directory
        print("Mode: Batch transcription")
        audio_directory = "audio_files"  # Change this to your directory

        if os.path.exists(audio_directory):
            transcribe_directory(audio_directory, model_size= ukuran_model, output_format="both")
        else:
            print(f"Please create a directory '{audio_directory}' and place your MP3 files there,")
            print(f"or update the 'audio_file' or 'audio_directory' variables in the script.")

    print("\n=== Transcription Complete ===")

if __name__ == "__main__":
    main()

=== Indonesian Audio Transcription ===

Mode: Single file transcription
Loading Whisper model (large)...
Transcribing: 15.mp3
This may take a few minutes depending on the audio length...
[00:00.000 --> 00:07.580]  Mahasiswa baru mengikuti pekan orientasi minggu lalu.
✓ Transcription saved to: transcriptions/15_20260126_010722.txt
✓ Detailed transcription saved to: transcriptions/15_20260126_010722.json

=== Transcription Complete ===


In [ ]:
from google.colab import files
import os

# Download all transcription files
for filename in os.listdir('transcriptions'):
    files.download(f'transcriptions/{filename}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Clear folder


In [ ]:
import shutil
import os

# Hapus folder transcriptions lama dan buat yang baru
if os.path.exists('transcriptions'):
    shutil.rmtree('transcriptions')
    print("🗑️ File transcription lama dihapus")

os.makedirs('transcriptions', exist_ok=True)
print("✓ Siap untuk transcription baru!")

🗑️ File transcription lama dihapus
✓ Siap untuk transcription baru!
